In [3]:
# Import Necessary Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import sklearn
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import (confusion_matrix, 
                             roc_auc_score, 
                             average_precision_score)
from sklearn.model_selection import cross_val_score
import warnings
import time
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning, module="distributed.utils_perf")
from multiprocessing import Pool
import dask.dataframe as dd
from dask.distributed import Client
from dask.diagnostics import ProgressBar
from collections import Counter
import os
import re



year_quarter_pairs = [
    (2021, 1), (2021, 2), (2021, 3), (2021, 4),
    (2022, 1), (2022, 2), (2022, 3), (2022, 4),
    (2023, 1), (2023, 2), (2023, 3), (2023, 4),
    (2024, 1), (2024, 2), (2024, 3), (2024, 4),
    (2025, 1), (2025, 2)  # Only up to Q3 available
]


df_data_dict = pd.read_excel(r"C:\dwaraka\github projects\Risk Analytics\Freddie_mac_raw_data\Standard_Quaterly\file_layout.xlsx", 
                             sheet_name='Monthly Performance Data File', 
                             header=1)



cols_to_del = ['Non MI Recoveries', 'Net Sales Proceeds', 
                'Delinquent Accrued Interest', 'Actual Loss Calculation',
                'Miscellaneous Expenses', 'Taxes and Insurance', 
                'Maintenance and Preservation Costs', 'Legal Costs', 
                'Expenses', 'MI Recoveries', 'Step Modification Flag', 
                'Modification Flag', 
                'Deferred Payment Plan', 'Modification Cost', 
                'Current Month Modification Cost', 'Borrower Assistance Status Code',
                'Due Date of Last Paid Installment (DDLPI)', 
                'Zero Balance Removal UPB', 
                'Zero Balance Effective Date', 
                'Current Deferred UPB',
                ]



dtypes = {
    'Loan Sequence Number': 'object',
    'Monthly Reporting Period': 'str',
    'Current Actual UPB': 'float64',
    'Current Loan Delinquency Status': 'object',
    'Loan Age': 'float32', # Changed to float due to potential NaN values
    'Remaining Months to Legal Maturity': 'float32', # Changed to float due to potential NaN values
    'Defect Settlement Date': 'str',
    'Modification Flag': 'object',
    'Zero Balance Code': 'float32',
    'Zero Balance Effective Date': 'str',
    'Current Interest Rate': 'float64',
    'Current Deferred UPB': 'float64',
    'Due Date of Last Paid Installment (DDLPI)': 'str',
    'MI Recoveries': 'float64',
    'Net Sales Proceeds': 'object', # 'Alpha-Numeric' but the range is not specified
    'Non MI Recoveries': 'float64',
    'Expenses': 'float64',
    'Legal Costs': 'float64',
    'Maintenance and Preservation Costs': 'float64',
    'Taxes and Insurance': 'float64',
    'Miscellaneous Expenses': 'float64',
    'Actual Loss Calculation': 'float64',
    'Modification Cost': 'float64',
    'Step Modification Flag': 'object',
    'Deferred Payment Plan': 'object',
    'Estimated Loan-to-Value (ELTV)': 'float32', # Changed to float due to potential NaN values
    'Zero Balance Removal UPB': 'float64',
    'Delinquent Accrued Interest': 'float64',
    'Delinquency Due to Disaster': 'object',
    'Borrower Assistance Status Code': 'object',
    'Current Month Modification Cost': 'float64',
    'Interest Bearing UPB': 'float64'
}


COLUMN_NAMES = df_data_dict['ATTRIBUTE NAME'].values
cols_to_read = list(set(COLUMN_NAMES) - set(cols_to_del))
column_indices = [list(COLUMN_NAMES).index(col) for col in cols_to_read]




# Initialization
paid_loans_indices = set()
charged_loans_indices = set()
aggregated_dataframes = []

data_path_template = r"C:\dwaraka\github projects\Risk Analytics\Freddie_mac_raw_data\Standard_Quaterly\historical_data_{0}Q{1}\historical_data_time_{0}Q{1}.txt"

# Start the overall timer
start_time_overall = time.time()

for year, quarter in year_quarter_pairs:
        # Start the timer for this specific quarter
    start_time = time.time()
    
    data_path = data_path_template.format(year, quarter)
    
    # ✅ Add the file existence check here
    if not os.path.exists(data_path):
        print(f"⚠️ Missing file: {data_path}, skipping.")
        continue

    print('Processing ', year, 'Q', quarter)
    
    # Read only necessary columns
    monthly_data = pd.read_csv(
        data_path, 
        sep='|', 
        header=None, 
        names=df_data_dict['ATTRIBUTE NAME'].values,
        dtype=dtypes,
        usecols=column_indices
    )

    # Charged indices
    charged_loans_this_month = set(monthly_data.loc[monthly_data['Zero Balance Code'].isin([2.0, 3.0, 9.0]), 'Loan Sequence Number'].values)
    paid_loans_this_month = set(monthly_data.loc[monthly_data['Zero Balance Code'] == 1.0, 'Loan Sequence Number'].values)
    
    sampled_paid_indices = set(np.random.choice(list(paid_loans_this_month), len(charged_loans_this_month), replace=False))
    
    paid_loans_indices.update(sampled_paid_indices)
    charged_loans_indices.update(charged_loans_this_month)
    
    aggregated_dataframes.append(monthly_data[monthly_data['Loan Sequence Number'].isin(sampled_paid_indices.union(charged_loans_this_month))])
    
    del monthly_data

# Print overall processing time
print(f"\nTotal processing time: {time.time() - start_time_overall:.2f} seconds\n")


# Improved print statements
print(f"Total Paid Loans Indices: {len(paid_loans_indices)}")
print(f"Total Charged Loans Indices: {len(charged_loans_indices)}")
print(f"Total Selected Loan Indices: {len(paid_loans_indices) + len(charged_loans_indices)}")

df_monthly = pd.concat(aggregated_dataframes, ignore_index=True)

output_path = r"C:\dwaraka\github projects\Risk Analytics\Freddie_mac_raw_data\Standard_Quaterly\sampled_monthly_perf_2021_2025Q2.csv"
df_monthly.to_csv(output_path, index=False)
print(f"✅ Saved: {output_path}")


Processing  2021 Q 1
Processing  2021 Q 2
Processing  2021 Q 3
Processing  2021 Q 4
Processing  2022 Q 1
Processing  2022 Q 2
Processing  2022 Q 3
Processing  2022 Q 4
Processing  2023 Q 1
Processing  2023 Q 2
Processing  2023 Q 3
Processing  2023 Q 4
Processing  2024 Q 1
Processing  2024 Q 2
Processing  2024 Q 3
Processing  2024 Q 4
Processing  2025 Q 1
Processing  2025 Q 2

Total processing time: 584.12 seconds

Total Paid Loans Indices: 987
Total Charged Loans Indices: 987
Total Selected Loan Indices: 1974
✅ Saved: C:\dwaraka\github projects\Risk Analytics\Freddie_mac_raw_data\Standard_Quaterly\sampled_monthly_perf_2021_2025Q2.csv


In [4]:
df_data_dict_orig = pd.read_excel(r"C:\dwaraka\github projects\Risk Analytics\Freddie_mac_raw_data\Standard_Quaterly\file_layout.xlsx",
                             sheet_name='Origination Data File', 
                             header=1)


dtypes_orig = {
    'Credit Score': 'float64',
    'First Payment Date': 'str',
    'First Time Homebuyer Flag': 'object',
    'Maturity Date': 'str',
    'Metropolitan Statistical Area (MSA) Or Metropolitan Division': 'float64',
    'Mortgage Insurance Percentage (MI %)': 'float64',
    'Number of Units': 'float64',
    'Occupancy Status': 'object',
    'Original Combined Loan-to-Value (CLTV)': 'float64',
    'Original Debt-to-Income (DTI) Ratio': 'float64',
    'Original UPB': 'float64',
    'Original Loan-to-Value (LTV)': 'float64',
    'Original Interest Rate': 'float64',
    'Channel': 'object',
    'Prepayment Penalty Mortgage (PPM) Flag': 'object',
    'Amortization Type (Formerly Product Type)': 'object',
    'Property State': 'object',
    'Property Type': 'object',
    'Postal Code': 'float64',
    'Loan Sequence Number': 'object',
    'Loan Purpose': 'object',
    'Original Loan Term': 'float64',
    'Number of Borrowers': 'float64',
    'Seller Name': 'object',
    'Servicer Name': 'object',
    'Super Conforming Flag': 'object',
    'Pre-HARP Loan Sequence Number': 'object',
    'Program Indicator': 'object',
    'HARP Indicator': 'object',
    'Property Valuation Method': 'float64',
    'Interest Only (I/O) Indicator': 'object',
    'Mortgage Insurance Cancellation Indicator': 'object'
}

# Initialization
aggregated_dataframes_orig = []

data_path_template_orig = r"C:\dwaraka\github projects\Risk Analytics\Freddie_mac_raw_data\Standard_Quaterly\historical_data_{0}Q{1}\historical_data_{0}Q{1}.txt"


for year, quarter in year_quarter_pairs:
        # Start the timer for this specific quarter
    start_time = time.time()
    
    data_path = data_path_template_orig.format(year, quarter)
    
    # ✅ Add the file existence check here
    if not os.path.exists(data_path):
        print(f"⚠️ Missing file: {data_path}, skipping.")
        continue

    print('Processing ', year, 'Q', quarter)
    
    # Read only necessary columns
    orig_data = pd.read_csv(data_path, 
                               sep='|', 
                               header=None, 
                               names=df_data_dict_orig['ATTRIBUTE NAME'].values,
                               dtype=dtypes_orig)
    
    aggregated_dataframes_orig.append(pd.concat([
        orig_data[orig_data['Loan Sequence Number'].isin(paid_loans_indices)].reset_index(drop=True),
        orig_data[orig_data['Loan Sequence Number'].isin(charged_loans_indices)].reset_index(drop=True)
    ]))
    
    del orig_data
        
        

# Print overall processing time
print(f"\nTotal processing time: {time.time() - start_time_overall:.2f} seconds\n")

df_orig = pd.concat(aggregated_dataframes_orig, ignore_index=True)



# Define the output path
output_path_orig = r"C:\dwaraka\github projects\Risk Analytics\Freddie_mac_raw_data\Standard_Quaterly\sampled_origination_2021_2025Q2.csv"

# Save the DataFrame
df_orig.to_csv(output_path_orig, index=False)

print(f"✅ Saved: {output_path_orig}")

Processing  2021 Q 1
Processing  2021 Q 2
Processing  2021 Q 3
Processing  2021 Q 4
Processing  2022 Q 1
Processing  2022 Q 2
Processing  2022 Q 3
Processing  2022 Q 4
Processing  2023 Q 1
Processing  2023 Q 2
Processing  2023 Q 3
Processing  2023 Q 4
Processing  2024 Q 1
Processing  2024 Q 2
Processing  2024 Q 3
Processing  2024 Q 4
Processing  2025 Q 1
Processing  2025 Q 2

Total processing time: 650.22 seconds

✅ Saved: C:\dwaraka\github projects\Risk Analytics\Freddie_mac_raw_data\Standard_Quaterly\sampled_origination_2021_2025Q2.csv
